# Calculating Historical Performance for Unique Flights (An Optimized Approach)

Each row in the flights dataset represents a unique flight, containing details such as flight date, carrier, origin, destination, scheduled departure time, arrival time, and performance metrics like delays and cancellations. To extract meaningful historical insights for each flight, I performed rolling window calculations to gather statistics based on recent flight history over a certain period of time. The idea for this aggregation approach to create new features was adapted from [Ashish Jain](https://github.com/aajains/springboard-datascience-intensive/blob/master/capstone_project/DataAcquisitionMerging_v1/data_acquisition_merging.ipynb).

#### Objective:
I wanted to compute historical performance statistics—such as delays, cancellations, and diversions—for each flight, using its recent history. For example, for a flight from Las Vegas (LAS) to Charlotte (CLT) on August 6th, 2023, at 11:59 PM, I wanted to know how many times a flight on the same route, operated by the same airline, and within the same time window, was delayed or canceled over the past 10 days, 20 days, 30 days and 90 days.

#### Key Variables Defining a Unique Flight:
1. **Carrier**: The airline operating the flight.
2. **Origin**: The departure airport.
3. **Destination**: The arrival airport.
4. **Departure Window**: A specific time range during which the flight departs (e.g., morning, afternoon).

For each flight, I gathered historical data based on these four variables.

#### Process Overview:
1. **Grouping by Flight Attributes**: First step is to group flights by their carrier, route (origin-destination pair), and departure window. 
    * Use the function `create_route_ids()` to create a new dataframe containing unique routes made up of origin destination pairs and assign each route a `route_id`. This id was then added to each flight in the flights dataframe. 
    * Use the function `create_time_windows()` to create a new categorical column called `departure_window` with the categories and times below. These windows are based on the scheduled departure time in the local timezone. 

        | Category         | Time Range    |
        |------------------|---------------|
        | Overnight        | 12 AM - 4 AM  |
        | Early morning    | 4 AM - 6 AM   |
        | Morning          | 6 AM - 11 AM  |
        | Midday           | 11 AM - 1 PM  |
        | Early afternoon  | 1 PM - 3 PM   |
        | Afternoon        | 3 PM - 5 PM   |
        | Evening          | 5 PM - 7 PM   |
        | Night            | 7 PM - 10 PM  |
        | Late night       | 10 PM - 12 AM |


   
2. **Rolling Window Aggregations**: Next, for each flight, use the `calculate_flight_performance_aggregations(df, windows)` function calculate performance statistics (e.g., delay metrics, cancellations, and diversions) over rolling time windows (e.g., 10, 20, 30, and 90 days) that look back from the flight's scheduled departure date. The statistics include:
   - Median and maximum departure/arrival delays
   - Count of canceled flights
   - Count of diversions
   - Count of flights

3. **Handling Duplicate Data**: The aggregations are done for all specified time windows for each flight in the flights dataset. Some duplicates occurred when an airline had multiple flights on the same route and in the same time window. This caused the merge with the original flights dataset to result in more rows than expected, as one flight could be matched with multiple rows of aggregated data. After computing the rolling window statistics, I optimized the deduplication process by sorting the aggregated dataframe by key columns (route_id, airline_mkt, departure_window, scheduled_departure_datetime) and keeping the row with the highest number of n_flights. This ensured that for each combination of key columns, the row with the most complete flight data (i.e., the highest number of flights) is retained.

This deduplication was applied with the function, `drop_agg_duplicates()` before merging the rolling statistics back into the original flights dataframe.

4. **Time Windows**: Flights departing at similar times are grouped into “time windows” (e.g., early morning, afternoon), allowing for tracking consistent patterns in flight operations over time. This reduces the granularity of individual departure times and simplifies grouping.

#### Optimized Approach:
Rather than looping through each flight row by row, I leveraged vectorized operations with `pandas` to group flights by carrier, route, and departure window and applied rolling windows efficiently. This dramatically reduces computation time and allows the process to effectively scale large datasets with millions of rows. After calculating the aggregations, I merged the results back into the original flight dataset.

By combining these rolling window statistics with the original flight data, I enhanced the dataset with valuable historical features that can be used for further analysis and modeling.

#### Future Steps for EDA
1. **Handling Rows with `n_flights == 0` or `n_flights == NaN`**

    During the rolling window aggregation process, some rows resulted in `n_flights == 0`, indicating that there were no relevant flights within the specified window for the given route, carrier, and departure window. This can happen for several reasons:
    * Sparse Data: Some routes or carriers may not have had enough flights during the rolling window, especially for less common routes or carriers.
    * Edge Cases: Flights near the start or end of the dataset may not have enough prior or subsequent data to fill the rolling window (example flights in the first 10 days of the year)

    Rows with `n_flights == 0` often have NaN values for other aggregated columns (e.g., delays or cancellations), as no data was available to compute these statistics.

    *Handling Strategy:*
    * Preserving Data: Rather than dropping rows with `n_flights == 0`, I preserved them in the dataset. These rows will be handled during Exploratory Data Analysis (EDA) or preprocessing steps, where I can choose to filter them out, impute missing values, or flag them for special handling, depending on the analysis needs.
    * Analysis Consideration: These rows are important to consider because they reflect scenarios where there is limited historical data for specific flights, which can impact the accuracy of predictions or analysis. By keeping them, I maintain transparency in the data and can address any gaps during the modeling process.

In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

In [2]:
import pandas as pd
import numpy as np

from config.config import DATA_PATH


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
flights = pd.read_csv(DATA_PATH + '/interim/2023-performance-data-clean.csv',  parse_dates=['scheduled_departure_datetime'])

In [ ]:
flights.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0
4,2023,3,8,12,6,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,4.0,0.0,0.0,18.0,6.0,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-12 23:10:00,2023-08-13 06:28:00,2023-08-12 23:14:00,2023-08-13 06:24:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-13 06:00:00+00:00,2023-08-13 06:14:00+00:00,2023-08-13 10:00:00+00:00,2023-08-13 10:24:00+00:00,0


In [6]:
flights.shape

(7260941, 56)

In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7260941 entries, 0 to 7260940
Data columns (total 56 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   year                              int64         
 1   quarter                           int64         
 2   month                             int64         
 3   day_of_month                      int64         
 4   day_of_week                       int64         
 5   marketing_airline_id              int64         
 6   flight_number_marketing_airline   int64         
 7   operating_airline_id              int64         
 8   tail_number                       object        
 9   origin_airport_id                 int64         
 10  origin_city_market_id             int64         
 11  origin                            object        
 12  origin_state                      object        
 13  dest_airport_id                   int64         
 14  dest_city_market_i

In [8]:
def create_route_ids(df):
    """
    Assign unique route IDs to each origin and destination airport pair.

    Parameters:
    ----------
    flights : pandas.DataFrame
        The original DataFrame of flight data, which must contain 'origin' and 'dest' columns.

    Returns:
    -------
    flights : pandas.DataFrame
        The DataFrame with two new columns:
        - 'route_id': A unique identifier for each route (origin-destination pair).
    """

    routes = df.groupby(['origin', 'dest'])[['origin', 'dest']].size().reset_index(name='flight_count')
    routes.index += 1
    routes = routes.reset_index().rename(columns={'index':'route_id'})

    # create a dictionary with route tuples and route id
    route_mapping = routes.set_index(['origin', 'dest'])['route_id'].to_dict()

    # add the route id for each flight
    df['route_id'] = df[['origin', 'dest']].apply(tuple, axis=1).map(route_mapping)

    return df  

In [9]:
flights = create_route_ids(flights)

In [10]:
def create_time_windows(df, datetime_col):
    """
    Categorize flights into departure time windows.

    Time windows (e.g., morning, afternoon) are assigned based on the hour of day using 
    predefined bins.

    Parameters:
    ----------
    flights : pandas.DataFrame
        The original DataFrame of flight data, which must contain a datetime column to 
        convert such as the 'scheduled_departure_datetime' column.

    Returns:
    -------
    flights : pandas.DataFrame
        The DataFrame with two new columns:
        - 'departure_window': A categorical time window label (e.g., morning, afternoon).
    """

    bins = [0, 4, 6, 11, 13, 15, 17, 19, 22, 24]
    labels = ['overnight', 'early morning', 'morning', 'midday', 'early afternoon', 'afternoon', 'evening', 'night', 'late night']

    df['hour_of_day'] = df[datetime_col].dt.hour

    df['departure_window'] = pd.cut(df['hour_of_day'], bins=bins, labels=labels)

    return df

In [11]:
flights = create_time_windows(flights, 'scheduled_departure_datetime')

In [12]:
flights.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,route_id,hour_of_day,departure_window
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,3228,23,late night
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,3228,23,late night
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,3228,23,late night
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0,3228,23,late night
4,2023,3,8,12,6,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,4.0,0.0,0.0,18.0,6.0,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-12 23:10:00,2023-08-13 06:28:00,2023-08-12 23:14:00,2023-08-13 06:24:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-13 06:00:00+00:00,2023-08-13 06:14:00+00:00,2023-08-13 10:00:00+00:00,2023-08-13 10:24:00+00:00,0,3228,23,late night


In [13]:
def calculate_flight_performance_aggregations(df, windows):
    """
    Calculate rolling window flight performance aggregations over specified time windows.

    This function calculates statistics like mean, median, max of departure and arrival delays,
    as well as the number of cancellations, diversions, and the number of flights, for each 
    flight based on its carrier, route, and departure window.

    Parameters:
    ----------
    df : pandas.DataFrame
        The input DataFrame containing flight data, including columns such as 'airline_mkt', 
        'route_id', 'departure_window', and 'scheduled_departure_datetime'.
        
    windows : list of str
        A list of rolling window durations (e.g., ['10D', '20D', '30D']) to calculate the
        rolling aggregations over. These should be formatted as Pandas-compatible time strings.

    Returns:
    -------
    pandas.DataFrame
        A DataFrame containing the rolling statistics for each window. Each statistic will be 
        suffixed with the corresponding window (e.g., 'dep_delay_mean_10D', 'n_flights_20D').
        The resulting DataFrame includes statistics like:
        - dep_delay_mean, dep_delay_median, dep_delay_max
        - arr_delay_mean, arr_delay_median, arr_delay_max
        - cancelled_sum, div_airport_landings_sum
        - n_flights (number of flights in the window)
    """
    # Sort dataframe by scheduled departure datetime column
    df = df.sort_values('scheduled_departure_datetime')

    # Group dataframe to create unique flights based on airline, route, and departure window
    grouped_df = df.groupby(['airline_mkt', 'route_id', 'departure_window'], observed=False)

    # Initiate empty list to capture aggregate values
    rolling_stats_list = []

    # Iterate through unique flight groupings to calculate performance metrics over rolling window
    for window in windows:
        rolling_stats = grouped_df.rolling(window=window, on='scheduled_departure_datetime').agg({
        'dep_delay': ['mean', 'median', 'max'],
        'arr_delay': ['mean', 'median', 'max'],
        'cancelled': 'sum',
        'div_airport_landings': 'sum'
        }).reset_index()

        # Add the number of flights by counting rows within each rolling window
        rolling_stats[f'n_flights_{window}'] = grouped_df.rolling(window=window, on='scheduled_departure_datetime')['dep_delay'].count().values

        # Flatten the multi-level column index for easier access
        rolling_stats.columns = [
            f'{col[0]}_{col[1]}_{window}' if isinstance(col, tuple) and col[1] != '' else col[0] for col in rolling_stats.columns
        ]

        rolling_stats_list.append(rolling_stats)

    # Concatenate all rolling stats into a single DataFrame
    all_rolling_stats = pd.concat(rolling_stats_list, axis=1)

    # Remove duplicate columns from the concatenation (from the index or key columns)
    all_rolling_stats = all_rolling_stats.loc[:, ~all_rolling_stats.columns.duplicated()]

    return all_rolling_stats    


In [14]:
# calculate rolling statistics for 2023 unique flights
rolling_stats = calculate_flight_performance_aggregations(flights, ['10D', '30D', '90D'])

In [15]:
rolling_stats

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
0,Alaska Airlines Inc.,35,morning,2023-04-22 11:40:00,-1.000000,-1.0,-1.0,-10.000000,-10.0,-10.0,0.0,0.0,1.0,-1.000000,-1.0,-1.0,-10.000000,-10.0,-10.0,0.0,0.0,1.0,-1.000000,-1.0,-1.0,-10.000000,-10.0,-10.0,0.0,0.0,1.0
1,Alaska Airlines Inc.,35,morning,2023-04-23 11:40:00,-7.500000,-7.5,-1.0,-17.500000,-17.5,-10.0,0.0,0.0,2.0,-7.500000,-7.5,-1.0,-17.500000,-17.5,-10.0,0.0,0.0,2.0,-7.500000,-7.5,-1.0,-17.500000,-17.5,-10.0,0.0,0.0,2.0
2,Alaska Airlines Inc.,35,morning,2023-04-29 11:40:00,-7.333333,-7.0,-1.0,-15.666667,-12.0,-10.0,0.0,0.0,3.0,-7.333333,-7.0,-1.0,-15.666667,-12.0,-10.0,0.0,0.0,3.0,-7.333333,-7.0,-1.0,-15.666667,-12.0,-10.0,0.0,0.0,3.0
3,Alaska Airlines Inc.,35,morning,2023-04-30 11:40:00,-7.750000,-8.0,-1.0,-18.250000,-18.5,-10.0,0.0,0.0,4.0,-7.750000,-8.0,-1.0,-18.250000,-18.5,-10.0,0.0,0.0,4.0,-7.750000,-8.0,-1.0,-18.250000,-18.5,-10.0,0.0,0.0,4.0
4,Alaska Airlines Inc.,35,morning,2023-05-06 11:40:00,-5.666667,-7.0,-1.0,-20.000000,-22.0,-12.0,0.0,0.0,3.0,-6.400000,-7.0,-1.0,-19.000000,-22.0,-10.0,0.0,0.0,5.0,-6.400000,-7.0,-1.0,-19.000000,-22.0,-10.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249990,United Air Lines Inc.,6708,early afternoon,2023-12-27 15:15:00,13.181818,-3.0,63.0,-1.181818,-16.0,62.0,0.0,0.0,11.0,2.806452,-5.0,63.0,-11.838710,-16.0,62.0,0.0,0.0,31.0,3.126437,-4.0,104.0,-7.505747,-13.0,91.0,2.0,0.0,87.0
7249991,United Air Lines Inc.,6708,early afternoon,2023-12-28 15:15:00,7.000000,-3.0,60.0,-4.909091,-16.0,62.0,0.0,0.0,11.0,1.645161,-5.0,63.0,-12.838710,-19.0,62.0,0.0,0.0,31.0,3.034091,-4.0,104.0,-7.636364,-13.0,91.0,2.0,0.0,88.0
7249992,United Air Lines Inc.,6708,early afternoon,2023-12-29 15:15:00,6.818182,-4.0,60.0,-5.181818,-16.0,62.0,0.0,0.0,11.0,2.032258,-5.0,63.0,-12.903226,-19.0,62.0,0.0,0.0,31.0,3.079545,-4.0,104.0,-7.818182,-13.5,91.0,2.0,0.0,88.0
7249993,United Air Lines Inc.,6708,early afternoon,2023-12-30 15:15:00,7.727273,-4.0,60.0,-4.363636,-16.0,62.0,0.0,0.0,11.0,2.096774,-5.0,63.0,-12.612903,-19.0,62.0,0.0,0.0,31.0,3.090909,-4.0,104.0,-7.931818,-14.0,91.0,2.0,0.0,88.0


In [16]:
np.sort(rolling_stats['n_flights_10D'].unique())

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94.])

In [17]:
mask = rolling_stats['n_flights_10D'] == 0
rolling_stats[mask]

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
386,Alaska Airlines Inc.,39,afternoon,2023-01-01 16:15:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
822,Alaska Airlines Inc.,63,early afternoon,2023-11-18 14:25:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.666667,-18.0,43.0,-11.666667,-27.0,26.0,2.0,0.0,3.0
20956,Alaska Airlines Inc.,379,afternoon,2023-01-02 16:50:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
39118,Alaska Airlines Inc.,802,midday,2023-04-01 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
54610,Alaska Airlines Inc.,2067,early afternoon,2023-01-01 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7218902,United Air Lines Inc.,6401,early afternoon,2023-01-01 14:12:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
7226614,United Air Lines Inc.,6484,midday,2023-07-09 13:53:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,10.920000,-2.0,194.0,3.820000,-10.0,176.0,2.0,0.0,50.0
7237712,United Air Lines Inc.,6569,midday,2023-01-01 12:12:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
7238564,United Air Lines Inc.,6580,midday,2023-01-04 12:54:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0


In [18]:
rolling_stats.shape

(7249995, 31)

Initially I merged `rolling_stats` with the flights dataframe but due to duplicates, the merged dataframe was several thousand rows longer. The code below is how I investigated the duplicates. 

In [19]:
rolling_stats.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime']).sum()

152

In [20]:
rolling_stats[rolling_stats.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], keep=False)].sort_values('scheduled_departure_datetime').head()


,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
3832991,Frontier Airlines Inc.,3781,early morning,2023-03-05 05:45:00,-0.300000,-2.0,20.0,-6.500000,-8.0,18.0,0.0,0.0,10.0,-0.666667,-3.0,39.0,-6.259259,-8.0,40.0,0.0,0.0,27.0,1.156863,-1.0,39.0,-5.490196,-8.0,40.0,0.0,0.0,51.0
3832992,Frontier Airlines Inc.,3781,early morning,2023-03-05 05:45:00,-0.181818,-2.0,20.0,-5.363636,-8.0,18.0,0.0,0.0,11.0,-0.607143,-2.5,39.0,-5.821429,-8.0,40.0,0.0,0.0,28.0,1.153846,-1.0,39.0,-5.269231,-7.5,40.0,0.0,0.0,52.0
2491814,Delta Air Lines Inc.,288,morning,2023-03-14 09:10:00,-0.100000,-1.0,11.0,-0.200000,-3.0,20.0,1.0,0.0,10.0,3.310345,-1.0,99.0,3.344828,-2.0,101.0,2.0,0.0,29.0,4.805556,-1.0,113.0,-2.458333,-4.0,104.0,2.0,0.0,72.0
2491813,Delta Air Lines Inc.,288,morning,2023-03-14 09:10:00,0.111111,0.0,11.0,0.888889,-3.0,20.0,1.0,0.0,9.0,3.500000,-1.0,99.0,3.821429,-1.0,101.0,2.0,0.0,28.0,4.901408,-1.0,113.0,-2.352113,-4.0,104.0,2.0,0.0,71.0
3670290,Delta Air Lines Inc.,6191,midday,2023-03-14 12:40:00,13.400000,2.0,104.0,0.200000,-10.5,114.0,1.0,0.0,10.0,10.275862,0.0,106.0,1.655172,-5.0,114.0,2.0,0.0,29.0,6.464789,-2.0,108.0,2.380282,-1.0,114.0,2.0,0.0,71.0


In order to address the issue of one flight getting paired with two unique flight values, I decided to drop duplicates and keep the last value. 

In [21]:
def drop_agg_duplicates(df):
    """
    Remove duplicate rows from the aggregated flight performance data while keeping the row 
    with the highest number of flights (n_flights) for each group.

    The DataFrame is sorted by 'route_id', 'airline_mkt', 'departure_window', and 
    'scheduled_departure_datetime', and duplicates are dropped based on these columns. 
    The row with the highest 'n_flights' for each group is retained.

    Parameters:
    ----------
    df : pandas.DataFrame
        The input DataFrame containing the rolling flight performance statistics, 
        including the columns 'route_id', 'airline_mkt', 'departure_window', 
        'scheduled_departure_datetime', and 'n_flights_<window>'.

    Returns:
    -------
    pandas.DataFrame
        A deduplicated DataFrame where only one row per unique combination of 'route_id', 
        'airline_mkt', 'departure_window', and 'scheduled_departure_datetime' is retained,
        specifically the row with the highest 'n_flights'.
    """
    df_sorted = df.sort_values( ['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime', 'n_flights_90D'], ascending=True)

    df_deduped = df_sorted.drop_duplicates(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], keep='last')

    return df_deduped

In [22]:
rolling_stats_deduped = drop_agg_duplicates(rolling_stats)

In [23]:
rolling_stats_deduped.shape

(7249843, 31)

In [24]:
flights_historical_performance = pd.merge(flights, rolling_stats_deduped, 
    on=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], 
    how='left')

In [25]:
flights_historical_performance.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,route_id,hour_of_day,departure_window,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,3228,23,late night,5.125000,2.0,31.0,-5.000000,-6.0,16.0,3.0,0.0,8.0,15.518519,5.0,82.0,4.923077,0.5,62.0,4.0,0.0,27.0,12.543860,2.0,154.0,-2.142857,-7.0,124.0,4.0,0.0,57.0
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,3228,23,late night,14.000000,2.0,76.0,4.714286,-6.0,69.0,3.0,0.0,8.0,17.740741,5.0,82.0,7.961538,5.0,69.0,4.0,0.0,27.0,13.637931,2.5,154.0,-0.894737,-7.0,124.0,4.0,0.0,58.0
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,3228,23,late night,12.375000,1.5,76.0,5.000000,-6.0,69.0,3.0,0.0,8.0,15.185185,5.0,82.0,6.192308,0.5,69.0,4.0,0.0,27.0,13.220339,2.0,154.0,-1.103448,-7.0,124.0,4.0,0.0,59.0
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0,3228,23,late night,10.111111,1.0,76.0,0.625000,-8.5,69.0,2.0,0.0,9.0,11.851852,3.0,76.0,2.653846,-0.5,69.0,4.0,0.0,27.0,12.866667,2.0,154.0,-1.593220,-7.0,124.0,4.0,0.0,60.0
4,2023,3,8,12,6,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,4.0,0.0,0.0,18.0

In [26]:
flights_historical_performance.shape[0] == flights.shape[0]

True

In [27]:
flights_historical_performance.to_csv(DATA_PATH + '/interim/flights_historical_performance.csv', index=False)